In [1]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments

from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

In [3]:
from huggingface_hub import notebook_login

notebook_login()  # same as before

In [2]:
# 0. imports
import os
from dataclasses import dataclass, field
from typing import Dict, Optional

import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from transformers import AutoTokenizer, HfArgumentParser, TrainingArguments

from trl import DPOTrainer

In [5]:
from datasets import load_dataset

#dataset = load_dataset("Dahoas/full-hh-rlhf",split='test')
dataset = load_dataset('json',data_files='mungeo_dp.jsonl',split = "train",use_auth_token=True)

In [6]:
dataset

Dataset({
    features: ['prompt', 'response', 'chosen', 'rejected'],
    num_rows: 231
})

In [7]:
import pandas as pd
Dataset = pd.read_csv('data_dpo.csv')

In [8]:
Dataset

,prompt,chosen,rejected
0,"[INST]<<SYS>>\n You are a helpful, respectful ...",The patient's diagnosis is chronic inflammator...,Primary Diagnosis: Chronic inflammatory demyel...
1,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the given information, it is mentione...",The patient's diagnosis of CIDP has been confi...
2,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the given information, it is stated t...",The patient's diagnosis of CIDP has been confi...
3,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the given medical context, there is n...",The patient has not tried conventional therapy...
4,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the provided context, there is no inf...",The patient's current weight in kilograms (kg)...
...,...,...,...
226,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the provided medical context, there i...",Given the provided medical context and the age...
227,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the provided context, it is indicated...",The patient's medical status does not require ...
228,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the provided context, it is not possi...",The patient is at risk for fluid overload and ...
229,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the provided medical context, there i...",The context provided does not contain any sign...


In [9]:
Dataset.to_json('dp0.json')

In [10]:
df = pd.read_json('dp0.json')
df.to_json("dpo.jsonl", orient="records", lines=True)

In [4]:
from datasets import load_dataset

#dataset = load_dataset("Dahoas/full-hh-rlhf",split='test')
dataset = load_dataset('json',data_files='dpo.jsonl',split = "train",use_auth_token=True)

/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


In [5]:
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 231
})

In [6]:
original_columns = dataset.column_names

def return_prompt_and_responses(samples):
    return {
        "prompt": samples['prompt'],
        "chosen": samples["chosen"],
        "rejected": samples["rejected"],
    }

In [7]:
train_data = dataset.map(return_prompt_and_responses,batched=True,remove_columns=original_columns)


In [8]:
train_data


Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 231
})

In [9]:
# 1. load a pretrained model
model = AutoPeftModelForCausalLM.from_pretrained("/home/jovyan/sai/sft_biomedgpt7b",low_cpu_mem_usage=True,torch_dtype=torch.float16,load_in_4bit=True)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
CUDA extension not installed.
CUDA extensi

In [10]:
# Load the reference model 
model_ref = AutoPeftModelForCausalLM.from_pretrained("/home/jovyan/sai/sft_biomedgpt7b",low_cpu_mem_usage=True,torch_dtype=torch.float16,load_in_4bit=True)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
model_ref

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False


In [12]:
tokenizer = AutoTokenizer.from_pretrained("/home/jovyan/sai/sft_biomedgpt7b")
tokenizer.pad_token = tokenizer.eos_token

In [13]:
train_df = train_data.to_pandas()


In [14]:
train_df.head()


,prompt,chosen,rejected
0,"[INST]<<SYS>>\n You are a helpful, respectful ...",The patient's diagnosis is chronic inflammator...,Primary Diagnosis: Chronic inflammatory demyel...
1,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the given information, it is mentione...",The patient's diagnosis of CIDP has been confi...
2,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the given information, it is stated t...",The patient's diagnosis of CIDP has been confi...
3,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the given medical context, there is n...",The patient has not tried conventional therapy...
4,"[INST]<<SYS>>\n You are a helpful, respectful ...","Based on the provided context, there is no inf...",The patient's current weight in kilograms (kg)...


In [15]:
val_df = train_df.sample(50)


In [16]:
train_data = dataset.from_pandas(train_df)
val_data = dataset.from_pandas(val_df)


In [17]:
peft_config = LoraConfig(
        r=4,
        lora_alpha=6,
        lora_dropout=0.1,
        target_modules=["q_proj", "k_proj","v_proj","o_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )
peft_config.inference_mode = False

In [19]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training


model = prepare_model_for_kbit_training(model)
model.config.use_cache=False
model.gradient_checkpointing_enable()
model.config.pretraining_tp=1
model = get_peft_model(model, peft_config)

In [20]:
model_ref = prepare_model_for_kbit_training(model_ref)
model_ref.config.use_cache=False
model_ref.gradient_checkpointing_enable()
model_ref.config.pretraining_tp=1
model_ref = get_peft_model(model_ref, peft_config)

In [21]:
training_args = TrainingArguments(
        per_device_train_batch_size=1,
        max_steps=250,
        remove_unused_columns=False,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        evaluation_strategy="steps",
        logging_first_step=True,
        logging_steps=10,
        output_dir="biomedgpt_dpo7b",
        optim="paged_adamw_32bit",
        warmup_steps=2,
        fp16=True,
        #push_to_hub=True
    )

In [22]:
dpo_trainer = DPOTrainer(
        model,
        model_ref,
        args=training_args,
        beta=0.1,
        train_dataset=train_data,
        eval_dataset=val_data,
        tokenizer=tokenizer,
        max_length=512,
        #max_target_length=256,
        max_prompt_length=256
    )

In [23]:
dpo_trainer.train()


/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
10,0.593700,0.456292,0.622710,0.068058,0.937500,0.554653,-49.000015,-64.745850,-1.909305,-1.926689
20,0.283300,0.252731,1.301233,-0.811575,0.937500,2.112808,-57.796345,-57.960617,-1.910291,-1.933762
30,0.091300,0.295203,0.558034,-3.664357,0.953125,4.222391,-86.324158,-65.392609,-1.858414,-1.893823
40,0.632400,0.212230,0.860482,-4.389436,0.937500,5.249918,-93.574951,-62.368130,-1.833933,-1.892643
50,0.225800,0.273557,0.197908,-7.588692,0.953125,7.786599,-125.567505,-68.993866,-1.826356,-1.892543
60,0.224600,0.489242,-3.986630,-19.241064,0.937500,15.254434,-242.091232,-110.839241,-1.707354,-1.770775
70,0.256400,0.496837,-3.818516,-24.964119,0.953125,21.145603,-299.321777,-109.158112,-1.678956,-1.795144
80,0.048700,1.413266,-8.814586,-44.175907,0.937500,35.361320,-491.439667,-159.118805,-1.651946,-1.803871
90,0.000000,3.738147,-20.301727,-66.685677,0.875000,46.383949,-716.537354,-273.990234,-1.558827,-1.692922
100,1.434800,1.087242,-6.913808,-40.785824,0.953125,33.872017,-457.538818,-140.111023,-1.765298,-1.897762


Token indices sequence length is longer than the specified maximum sequence length for this model (2275 > 2048). Running this sequence through the model will result in indexing errors


TrainOutput(global_step=250, training_loss=0.5321710271349732, metrics={'train_runtime': 2970.4836, 'train_samples_per_second': 0.168, 'train_steps_per_second': 0.084, 'total_flos': 0.0, 'train_loss': 0.5321710271349732, 'epoch': 2.16})

In [25]:
dpo_trainer.save_model("biomedgpt_dpo7b")
output_dir = os.path.join("biomedgpt_dpo7b", "final_checkpoint")
dpo_trainer.model.save_pretrained(output_dir)